In [2]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 1.25.0


In [1]:
from azureml.core import Workspace

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.
Workspace name: ml-service
Azure region: westus2
Subscription id: 3e0e14b3-7e28-4da7-97de-0f5cb324f030
Resource group: ml


### Create or Attach existing compute resource
By using Azure Machine Learning Compute, a managed service, data scientists can train machine learning models on clusters of Azure virtual machines. Examples include VMs with GPU support. In this tutorial, you create Azure Machine Learning Compute as your training environment. The code below creates the compute clusters for you if they don't already exist in your workspace.

**Creation of compute takes approximately 5 minutes. If the AmlCompute with that name is already in your workspace the code will skip the creation process.**

In [2]:
import os
from azureml.core.compute import AmlCompute, ComputeTarget

# choose a name for your cluster
compute_name = os.environ.get("AML_COMPUTE_CLUSTER_NAME", "ds3cluster")
compute_min_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MIN_NODES", 0)
compute_max_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MAX_NODES", 2)

# This example uses CPU VM. For using GPU VM, set SKU to STANDARD_NC6
vm_size = os.environ.get("AML_COMPUTE_CLUSTER_SKU", "STANDARD_DS3_V2")


if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print('found compute target. just use it. ' + compute_name)
else:
    print('creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = vm_size,
                                                                min_nodes = compute_min_nodes, 
                                                                max_nodes = compute_max_nodes)

    # create the cluster
    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)
    
    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
     # For a more detailed view of current AmlCompute status, use get_status()
    print(compute_target.get_status().serialize())

found compute target. just use it. ds3cluster


### Create a datastore containing sample images
The input dataset used for this notebook differs from a standard MNIST dataset in that it has been converted to PNG images to demonstrate use of files as inputs to Batch Inference. A sample of PNG-converted images of the MNIST dataset were take from [this repository](https://github.com/myleott/mnist_png).

We have created a public blob container `sampledata` on an account named `pipelinedata`, containing these images from the MNIST dataset. In the next step, we create a datastore with the name `images_datastore`, which points to this blob container. In the call to `register_azure_blob_container` below, setting the `overwrite` flag to `True` overwrites any datastore that was created previously with that name. 

This step can be changed to point to your blob container by providing your own `datastore_name`, `container_name`, and `account_name`.

In [3]:
from azureml.core.datastore import Datastore

account_name = "pipelinedata"
datastore_name = "mnist_datastore"
container_name = "sampledata"

mnist_data = Datastore.register_azure_blob_container(ws, 
                      datastore_name=datastore_name, 
                      container_name=container_name, 
                      account_name=account_name,
                      overwrite=True)

Next, let's specify the default datastore for the outputs.

In [4]:
def_data_store = ws.get_default_datastore()

### Create a FileDataset
A [FileDataset](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.data.filedataset?view=azure-ml-py) references single or multiple files in your datastores or public urls. The files can be of any format. FileDataset provides you with the ability to download or mount the files to your compute. By creating a dataset, you create a reference to the data source location. If you applied any subsetting transformations to the dataset, they will be stored in the dataset as well. The data remains in its existing location, so no extra storage cost is incurred.

You can use dataset objects as inputs. Register the datasets to the workspace if you want to reuse them later.

In [5]:
from azureml.core.dataset import Dataset

mnist_ds_name = 'mnist_sample_data'

path_on_datastore = mnist_data.path('mnist')
input_mnist_ds = Dataset.File.from_files(path=path_on_datastore, validate=False)

The input dataset can be specified as a pipeline parameter, so that you can pass in new data when rerun the PRS pipeline.

In [6]:
from azureml.data.dataset_consumption_config import DatasetConsumptionConfig
from azureml.pipeline.core import PipelineParameter

pipeline_param = PipelineParameter(name="mnist_param", default_value=input_mnist_ds)
input_mnist_ds_consumption = DatasetConsumptionConfig("minist_param_config", pipeline_param).as_mount()

### Intermediate/Output Data
Intermediate data (or output of a Step) is represented by [PipelineData](https://docs.microsoft.com/en-us/python/api/azureml-pipeline-core/azureml.pipeline.core.pipelinedata?view=azure-ml-py) object. PipelineData can be produced by one step and consumed in another step by providing the PipelineData object as an output of one step and the input of one or more steps.

In [7]:
from azureml.pipeline.core import Pipeline, PipelineData

output_dir = PipelineData(name="inferences", datastore=def_data_store)

### Download the Model

Download and extract the model from https://pipelinedata.blob.core.windows.net/mnist-model/mnist-tf.tar.gz to "models" directory

In [ ]:
import tarfile
import urllib.request

# create directory for model
model_dir = 'models'
if not os.path.isdir(model_dir):
    os.mkdir(model_dir)

url="https://pipelinedata.blob.core.windows.net/mnist-model/mnist-tf.tar.gz"
response = urllib.request.urlretrieve(url, "model.tar.gz")
tar = tarfile.open("model.tar.gz", "r:gz")
tar.extractall(model_dir)

os.listdir(model_dir)

### Register the model with Workspace
A registered model is a logical container for one or more files that make up your model. For example, if you have a model that's stored in multiple files, you can register them as a single model in the workspace. After you register the files, you can then download or deploy the registered model and receive all the files that you registered.

Using tags, you can track useful information such as the name and version of the machine learning library used to train the model. Note that tags must be alphanumeric. Learn more about registering models [here](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-deploy-and-where#registermodel) 

In [ ]:
from azureml.core.model import Model

# register downloaded model 
model = Model.register(model_path="models/",
                       model_name="mnist-prs", # this is the name the model is registered as
                       tags={'pretrained': "mnist"},
                       description="Mnist trained tensorflow model",
                       workspace=ws)

### Using your model to make batch predictions
To use the model to make batch predictions, you need an **entry script** and a list of **dependencies**:

#### An entry script
This script accepts requests, scores the requests by using the model, and returns the results.
- __init()__ - Typically this function loads the model into a global object. This function is run only once at the start of batch processing per worker node/process. Init method can make use of following environment variables (ParallelRunStep input):
    1.	AZUREML_BI_OUTPUT_PATH â€“ output folder path
- __run(mini_batch)__ - The method to be parallelized. Each invocation will have one minibatch.<BR>
__mini_batch__: Batch inference will invoke run method and pass either a list or Pandas DataFrame as an argument to the method. Each entry in min_batch will be - a filepath if input is a FileDataset, a Pandas DataFrame if input is a TabularDataset.<BR>
__run__ method response: run() method should return a Pandas DataFrame or an array. For append_row output_action, these returned elements are appended into the common output file. For summary_only, the contents of the elements are ignored. For all output actions, each returned output element indicates one successful inference of input element in the input mini-batch.
    User should make sure that enough data is included in inference result to map input to inference. Inference output will be written in output file and not guaranteed to be in order, user should use some key in the output to map it to input.
    

#### Dependencies
Helper scripts or Python/Conda packages required to run the entry script.

In [ ]:
scripts_folder = "Code"
script_file = "digit_identification.py"

# peek at contents
with open(os.path.join(scripts_folder, script_file)) as inference_file:
    print(inference_file.read())

## Build and run the batch inference pipeline
The data, models, and compute resource are now available. Let's put all these together in a pipeline.

###  Specify the environment to run the script
Specify the conda dependencies for your script. This will allow us to install pip packages as well as configure the inference environment.
* Always include **azureml-core** and **azureml-dataset-runtime\[fuse\]** in the pip package list to make ParallelRunStep run properly.

If you're using custom image (`batch_env.python.user_managed_dependencies = True`), you need to install the package to your image.

In [13]:
from azureml.core import Environment
from azureml.core.runconfig import CondaDependencies, DEFAULT_CPU_IMAGE

batch_conda_deps = CondaDependencies.create(pip_packages=["tensorflow==1.15.2", "pillow", 
                                                          "azureml-core", "azureml-dataset-runtime[fuse]"])
batch_env = Environment(name="batch_environment")
batch_env.python.conda_dependencies = batch_conda_deps
batch_env.docker.enabled = True
batch_env.docker.base_image = DEFAULT_CPU_IMAGE

'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


###  Create the configuration to wrap the inference script

In [14]:
from azureml.pipeline.core import PipelineParameter
from azureml.pipeline.steps import ParallelRunStep, ParallelRunConfig

parallel_run_config = ParallelRunConfig(
    source_directory=scripts_folder,
    entry_script=script_file,
    mini_batch_size=PipelineParameter(name="batch_size_param", default_value="5"),
    error_threshold=10,
    output_action="append_row",
    append_row_file_name="mnist_outputs.txt",
    environment=batch_env,
    compute_target=compute_target,
    process_count_per_node=PipelineParameter(name="process_count_param", default_value=2),
    node_count=2
)

### Create the pipeline step
Create the pipeline step using the script, environment configuration, and parameters. Specify the compute target you already attached to your workspace as the target of execution of the script. We will use ParallelRunStep to create the pipeline step.

In [15]:
parallelrun_step = ParallelRunStep(
    name="predict-digits-mnist",
    parallel_run_config=parallel_run_config,
    inputs=[ input_mnist_ds_consumption ],
    output=output_dir,
    allow_reuse=False
)

'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


### Run the pipeline
At this point you can run the pipeline and examine the output it produced. The Experiment object is used to track the run of the pipeline

In [16]:
from azureml.core import Experiment

pipeline = Pipeline(workspace=ws, steps=[parallelrun_step])
experiment = Experiment(ws, 'digit_identification')
pipeline_run = experiment.submit(pipeline)

Created step predict-digits-mnist [09691726][7dd94648-c082-4d96-abb3-6fc2463f963f], (This step will run and generate new outputs)
Submitted PipelineRun 1d217fb3-d9cd-4be9-a772-7490b7d75795
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/1d217fb3-d9cd-4be9-a772-7490b7d75795?wsid=/subscriptions/3e0e14b3-7e28-4da7-97de-0f5cb324f030/resourcegroups/ml/workspaces/ml-service&tid=72f988bf-86f1-41af-91ab-2d7cd011db47


### Monitor the run

The pipeline run status could be checked in Azure Machine Learning portal (https://ml.azure.com). The link to the pipeline run could be retrieved by inspecting the `pipeline_run` object.

In [17]:
# This will output information of the pipeline run, including the link to the details page of portal.
pipeline_run

Experiment,Id,Type,Status,Details Page,Docs Page
digit_identification,1d217fb3-d9cd-4be9-a772-7490b7d75795,azureml.PipelineRun,Preparing,Link to Azure Machine Learning studio,Link to Documentation


### Optional: View detailed logs (streaming) 

In [18]:
# Wait the run for completion and show output log to console
pipeline_run.wait_for_completion(show_output=True)

PipelineRunId: 1d217fb3-d9cd-4be9-a772-7490b7d75795
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/1d217fb3-d9cd-4be9-a772-7490b7d75795?wsid=/subscriptions/3e0e14b3-7e28-4da7-97de-0f5cb324f030/resourcegroups/ml/workspaces/ml-service&tid=72f988bf-86f1-41af-91ab-2d7cd011db47
PipelineRun Status: Running


StepRunId: 559bd0c1-d7d6-43f2-941b-dfe82f4e2b5e
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/559bd0c1-d7d6-43f2-941b-dfe82f4e2b5e?wsid=/subscriptions/3e0e14b3-7e28-4da7-97de-0f5cb324f030/resourcegroups/ml/workspaces/ml-service&tid=72f988bf-86f1-41af-91ab-2d7cd011db47
StepRun( predict-digits-mnist ) Status: NotStarted
StepRun( predict-digits-mnist ) Status: Queued

Streaming azureml-logs/20_image_build_log.txt
2021/03/29 08:29:47 Downloading source code...
2021/03/29 08:29:48 Finished downloading source code
2021/03/29 08:29:48 Creating Docker network: acb_default_network, driver: 'bridge'
2021/03/29 08:29:49 Successfully set up Docker network: acb_de

'Finished'

### View the prediction results per input image
In the digit_identification.py file above you can see that the ResultList with the filename and the prediction result gets returned. These are written to the DataStore specified in the PipelineData object as the output data, which in this case is called *inferences*. This containers the outputs from  all of the worker nodes used in the compute cluster. You can download this data to view the results ... below just filters to the first 10 rows

In [19]:
import pandas as pd
import tempfile

batch_run = pipeline_run.find_step_run(parallelrun_step.name)[0]
batch_output = batch_run.get_output_data(output_dir.name)

target_dir = tempfile.mkdtemp()
batch_output.download(local_path=target_dir)
result_file = os.path.join(target_dir, batch_output.path_on_datastore, parallel_run_config.append_row_file_name)

df = pd.read_csv(result_file, delimiter=":", header=None)
df.columns = ["Filename", "Prediction"]
print("Prediction has ", df.shape[0], " rows")
df.head(10) 

Prediction has  1000  rows


,Filename,Prediction
0,0.png,7
1,1.png,2
2,10.png,0
3,100.png,6
4,101.png,0
5,161.png,6
6,162.png,5
7,163.png,4
8,164.png,6
9,165.png,5


### Resubmit a with different dataset
Since we made the input a `PipelineParameter`, we can resubmit with a different dataset without having to create an entirely new experiment. We'll use the same datastore but use only a single image.

In [20]:
path_on_datastore = mnist_data.path('mnist/0.png')
single_image_ds = Dataset.File.from_files(path=path_on_datastore, validate=False)

In [21]:
pipeline_run_2 = experiment.submit(pipeline, 
                                   pipeline_parameters={"mnist_param": single_image_ds, 
                                                        "batch_size_param": "1",
                                                        "process_count_param": 1}
)

Submitted PipelineRun 87f4f4e1-4eea-4226-8ed3-4c50ac22d293
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/87f4f4e1-4eea-4226-8ed3-4c50ac22d293?wsid=/subscriptions/3e0e14b3-7e28-4da7-97de-0f5cb324f030/resourcegroups/ml/workspaces/ml-service&tid=72f988bf-86f1-41af-91ab-2d7cd011db47


In [22]:
# This will output information of the pipeline run, including the link to the details page of portal.
pipeline_run_2

Experiment,Id,Type,Status,Details Page,Docs Page
digit_identification,87f4f4e1-4eea-4226-8ed3-4c50ac22d293,azureml.PipelineRun,Preparing,Link to Azure Machine Learning studio,Link to Documentation


In [23]:
# Wait the run for completion and show output log to console
pipeline_run_2.wait_for_completion(show_output=True)

PipelineRunId: 87f4f4e1-4eea-4226-8ed3-4c50ac22d293
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/87f4f4e1-4eea-4226-8ed3-4c50ac22d293?wsid=/subscriptions/3e0e14b3-7e28-4da7-97de-0f5cb324f030/resourcegroups/ml/workspaces/ml-service&tid=72f988bf-86f1-41af-91ab-2d7cd011db47
PipelineRun Status: Running


StepRunId: 67fcd9f8-4f04-4e0b-841f-712296f8689b
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/67fcd9f8-4f04-4e0b-841f-712296f8689b?wsid=/subscriptions/3e0e14b3-7e28-4da7-97de-0f5cb324f030/resourcegroups/ml/workspaces/ml-service&tid=72f988bf-86f1-41af-91ab-2d7cd011db47
StepRun( predict-digits-mnist ) Status: Running

Streaming azureml-logs/55_azureml-execution-tvmps_11d6e793169222820dcef1afc63ff3c352409a69c79a9d308e9a7de38cffca56_d.txt
2021-03-29T08:50:56Z Starting output-watcher...
2021-03-29T08:50:56Z IsDedicatedCompute == True, won't poll for Low Pri Preemption
Login Succeeded
Using default tag: latest
latest: Pulling from azureml/azureml_1f3c26a22a

'Finished'

## Cleanup Compute resources

For re-occurring jobs, it may be wise to keep compute the compute resources and allow compute nodes to scale down to 0. However, since this is just a single-run job, we are free to release the allocated compute resources.

In [ ]:
# uncomment below and run if compute resources are no longer needed 
# compute_target.delete() 